In [1]:
print("Hello World")

Hello World


# Dataset 2

In [2]:
import os
import numpy as np
import cv2
import time
from sklearn.metrics import (
    accuracy_score, f1_score, confusion_matrix,
    precision_score, recall_score, mean_absolute_error,
    matthews_corrcoef, roc_auc_score, mean_squared_error
)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

data_dir = '/kaggle/input/brain-stroke-prediction-ct-scan-image-dataset/Brain_Stroke_CT-SCAN_image'
train_dir = os.path.join(data_dir, 'Train')
test_dir = os.path.join(data_dir, 'Test')
validation_dir = os.path.join(data_dir, 'Validation')

# Image dimensions
img_width, img_height = 256, 256

# Data augmentation and preprocessing
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Define metric lists
accuracies = []
precisions = []
recalls = []
specificities = []
f1_scores = []
maes = []
rmses = []
mccs = []
roc_aucs = []

# Step 2: Run Training and Evaluation 10 Times
for run in range(10):
    print(f"\n--- Run {run+1}/10 ---")
    
    # Reinitialize model for each run
    model = Sequential([
        Conv2D(8, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(32, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train
    model.fit(train_generator, epochs=100, validation_data=validation_generator, verbose=0)

    # Predict
    y_pred_prob = model.predict(test_generator)
    y_pred = (y_pred_prob > 0.5).astype(int).flatten()
    y_true = test_generator.classes

    # Metrics
    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred)
    rec = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    mcc = matthews_corrcoef(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred_prob)

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    spec = tn / (tn + fp)

    # Append to lists
    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    specificities.append(spec)
    f1_scores.append(f1)
    maes.append(mae)
    rmses.append(rmse)
    mccs.append(mcc)
    roc_aucs.append(roc_auc)

      # Print metrics for current run
    print(f"Accuracy:      {acc*100:.2f}")
    print(f"Precision:     {prec*100:.2f}")
    print(f"Recall:        {rec*100:.2f}")
    print(f"Specificity:   {spec*100:.2f}")
    print(f"F1 Score:      {f1*100:.2f}")
    print(f"MAE:           {mae:.4f}")
    print(f"RMSE:          {rmse:.4f}")
    print(f"MCC:           {mcc:.4f}")
    print(f"ROC AUC:       {roc_auc:.4f}")

# Final results
def mean_std(values):
    return np.mean(values), np.std(values)

model_name = "DNBSD"

# Compute mean ± std for each metric
acc_mean, acc_std = mean_std(accuracies)
prec_mean, prec_std = mean_std(precisions)
rec_mean, rec_std = mean_std(recalls)
spec_mean, spec_std = mean_std(specificities)
f1_mean, f1_std = mean_std(f1_scores)
mae_mean, mae_std = mean_std(maes)
rmse_mean, rmse_std = mean_std(rmses)
mcc_mean, mcc_std = mean_std(mccs)
roc_auc_mean, roc_auc_std = mean_std(roc_aucs)

# Final print
print(f"\nFinal Results after 10 runs — Model: {model_name}")
print(f"Accuracy:      {acc_mean*100:.2f} ± {acc_std*100:.2f}")
print(f"Precision:     {prec_mean*100:.2f} ± {prec_std*100:.2f}")
print(f"Recall:        {rec_mean*100:.2f} ± {rec_std*100:.2f}")
print(f"Specificity:   {spec_mean*100:.2f} ± {spec_std*100:.2f}")
print(f"F1 Score:      {f1_mean*100:.2f} ± {f1_std*100:.2f}")
print(f"MAE:           {mae_mean:.4f} ± {mae_std:.4f}")
print(f"RMSE:          {rmse_mean:.4f} ± {rmse_std:.4f}")
print(f"MCC:           {mcc_mean:.4f} ± {mcc_std:.4f}")
print(f"ROC AUC:       {roc_auc_mean:.4f} ± {roc_auc_std:.4f}")


2025-07-27 08:13:01.974293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753603982.339794      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753603982.441598      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Found 1843 images belonging to 2 classes.
Found 235 images belonging to 2 classes.
Found 437 images belonging to 2 classes.

--- Run 1/10 ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1753604003.389000      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1753604003.389717      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can i

14/14 ━━━━━━━━━━━━━━━━━━━━ 5s 320ms/step
Accuracy:      97.25
Precision:     92.14
Recall:        99.23
Specificity:   96.42
F1 Score:      95.56
MAE:           0.0275
RMSE:          0.1657
MCC:           0.9371
ROC AUC:       0.9983

--- Run 2/10 ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 155ms/step
Accuracy:      97.25
Precision:     92.14
Recall:        99.23
Specificity:   96.42
F1 Score:      95.56
MAE:           0.0275
RMSE:          0.1657
MCC:           0.9371
ROC AUC:       0.9993

--- Run 9/10 ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 156ms/step
Accuracy:      99.31
Precision:     97.74
Recall:        100.00
Specificity:   99.02
F1 Score:      98.86
MAE:           0.0069
RMSE:          0.0829
MCC:           0.9838
ROC AUC:       0.9999

--- Run 10/10 ---


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 159ms/step
Accuracy:      97.71
Precision:     92.86
Recall:        100.00
Specificity:   96.74
F1 Score:      96.30
MAE:           0.0229
RMSE:          0.1513
MCC:           0.9478
ROC AUC:       0.9986

Final Results after 10 runs — Model: DNBSD
Accuracy:      98.15 ± 0.71
Precision:     94.56 ± 2.00
Recall:        99.54 ± 0.51
Specificity:   97.56 ± 0.94
F1 Score:      96.98 ± 1.14
MAE:           0.0185 ± 0.0071
RMSE:          0.1330 ± 0.0289
MCC:           0.9572 ± 0.0162
ROC AUC:       0.9991 ± 0.0006
